In [1]:
import sys
import idx2numpy
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from load_datasets1 import load_data_from_csv

In [2]:
data_tr,data_val,data_test = load_data_from_csv(1)

# 数据加载
img_tr, lab_tr = data_tr
img_val, lab_val = data_val
img_test, lab_test = data_test

data_train = DataLoader(TensorDataset(img_tr, lab_tr), batch_size=64)
data_validation = DataLoader(TensorDataset(img_val, lab_val), batch_size=64)
data_test = DataLoader(TensorDataset(img_test, lab_test), batch_size=64)

In [48]:
## 定义神经网络
N = 5000
la = torch.rand([N,1],requires_grad=True)

class Inner_model(nn.Module):
    def __init__(self):
        super(Inner_model, self).__init__()
        self.fc = nn.Linear(28*28, 10)
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.fc(x)
        return x

net = Inner_model()

# 定义损失函数和优化器(以后的函数均只用一组数据，然后累加)

def lower_function(output, label, la):
    crossentropy = nn.CrossEntropyLoss()
    loss = crossentropy(output, label)*la
    return loss

def upper_function(output, label):
    crossentropy = nn.CrossEntropyLoss()
    loss = crossentropy(output, label)
    return loss

def inner_loop(net, data_train, lower_function, optimizer):
    running_loss = 0.0
    net.train()
    for i, (image, label) in enumerate(data_train):
        optimizer.zero_grad()

        hat_label = net(image)

        loss = lower_function(hat_label, label, la[i])
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

    return running_loss

def test(img_test,lab_test,net):
    correct = 0
    total = img_test.shape[0]
    with torch.no_grad():
        for i, image in enumerate(img_test, 0):
            label = lab_test[i]
            outputs = net(image)
        
            _, predicted = torch.max(outputs, 0)
            _, labels = torch.max(label,0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')



#optimizer = optim.Adam(net.parameters(), lr=1e-2)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
la_optimizer = optim.Adam([la], lr=1e-4)


test(img_test,lab_test,net)
# 训练网络
T = 1000

for epoch in range(T):
    running_loss1 = inner_loop(net, data_validation, lower_function, optimizer)

#    running_loss2 = 0.0
#    for j, (image, label) in enumerate(data_validation):

#        hat_label = net(image)

#        loss2 = upper_function(hat_label, label)
#        running_loss2 += loss2.item()

#        loss2.backward()

    if epoch % 20 == 0:
        print(f'loss1={running_loss1/(i+1)},loss2={running_loss2/(j+1)},epoch={epoch}')



# 测试网络
test(img_test,lab_test,net)

Accuracy of the network on the 10000 test images: 10.22 %
loss1=0.008514032891020179,loss2=6.408049553255491,epoch=0
loss1=0.007865327315917239,loss2=6.408049553255491,epoch=20
loss1=0.007627899385616184,loss2=6.408049553255491,epoch=40
loss1=0.007469156803470105,loss2=6.408049553255491,epoch=60
loss1=0.007349210473708808,loss2=6.408049553255491,epoch=80
loss1=0.0072529267221223565,loss2=6.408049553255491,epoch=100
loss1=0.007172756562475115,loss2=6.408049553255491,epoch=120
loss1=0.007104326590057462,loss2=6.408049553255491,epoch=140
loss1=0.007044849790353328,loss2=6.408049553255491,epoch=160
loss1=0.0069924296392593534,loss2=6.408049553255491,epoch=180
loss1=0.006945711846230552,loss2=6.408049553255491,epoch=200
loss1=0.006903692904440686,loss2=6.408049553255491,epoch=220
loss1=0.006865608109207824,loss2=6.408049553255491,epoch=240
loss1=0.006830860781762749,loss2=6.408049553255491,epoch=260
loss1=0.006798976554954424,loss2=6.408049553255491,epoch=280
loss1=0.006769572022464127,loss

In [30]:
a=la.grad/5000
print((a==0).sum())
print(la.shape)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [9]:
print(la.grad,la.grad.shape,la.shape)

tensor([[214.7339],
        [218.8986],
        [231.5886],
        ...,
        [  0.0000],
        [  0.0000],
        [  0.0000]]) torch.Size([5000, 1]) torch.Size([5000, 1])


In [16]:
print(net.fc.weight)
print(torch.norm(net.fc.weight.grad))

Parameter containing:
tensor([[-0.0024,  0.0348, -0.0058,  ..., -0.0250, -0.0014, -0.0282],
        [-0.0261, -0.0337, -0.0311,  ..., -0.0011, -0.0302, -0.0316],
        [-0.0220, -0.0117, -0.0098,  ..., -0.0216,  0.0117,  0.0028],
        ...,
        [-0.0266, -0.0191,  0.0122,  ..., -0.0072,  0.0172, -0.0201],
        [-0.0252,  0.0085,  0.0076,  ..., -0.0295,  0.0293, -0.0224],
        [-0.0151,  0.0024,  0.0032,  ...,  0.0022,  0.0018, -0.0167]],
       requires_grad=True)
tensor(17.1720)
